## Detecció automatitzada de defectes en teixits tècnics mitjançant visió artificial

### 1. Preparació de les dades
Carregar les imatges i crear còpies de la classe minoritària per equilibrar les dades entre les dues classes. Després, es realitza una divisió entre conjunt d'entrenament i conjunt de validació (80% - 20%) mitjançant la funció splitfolders.

In [ ]:
import os
import random
import shutil
import splitfolders 
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionResNetV2, InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input as preprocess_incep
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as preprocess_mobile
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras import applications
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import RMSprop, Adam
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
dir_defecte = "dataset_job1/1_defecte"
dir_no_defecte = "dataset_job1/0_nodefecte"

n_defecte = len(os.listdir(dir_defecte))
n_no_defecte = len(os.listdir(dir_no_defecte))
print(f"Defecte: {n_defecte}, No defecte: {n_no_defecte}")

# Quantes imatges calen per equilibrar
n_extra = n_defecte - n_no_defecte

# Llistar imatges de la classe minoritària
imatges = os.listdir(dir_no_defecte)

# Crear còpies
for i in range(n_extra):
    imatge_original = random.choice(imatges)
    nom_nou = f"copy_{i}_{imatge_original}"
    ruta_origen = os.path.join(dir_no_defecte, imatge_original)
    ruta_desti = os.path.join(dir_no_defecte, nom_nou)
    shutil.copy(ruta_origen, ruta_desti)

print(f"Imatges duplicades: {n_extra}")

In [ ]:
splitfolders.ratio("dataset_job1", output="dataset",
                   seed=1337, ratio=(.8, .2), move=False)

In [ ]:
print("Train set:")
print("defect -", len(os.listdir("dataset/train/1_defecte")))  
print("nodefect -", len(os.listdir("dataset/train/0_nodefecte")))
print("Validation set:")
print("defect -", len(os.listdir("dataset/val/1_defecte")))
print("nodefect -", len(os.listdir("dataset/val/0_nodefecte")))

### 2. Augmentació de les imatges
Per millorar la capacitat de generalització del model, augmentem les imatges amb rotacions aleatòries. Això ajuda a millorar el rendiment en la classificació.

In [ ]:
train_dir = "dataset/train"

angles = [90, 180, -90]       
augment_per_class = 495

def augmentar_classe(cls_dir, n_noves):
    originals = [f for f in os.listdir(cls_dir)
                 if not f.startswith('rot') and f.lower().endswith(('.jpg'))]
    generades = 0
    while generades < n_noves:
        nom_orig = random.choice(originals)
        ruta_orig = os.path.join(cls_dir, nom_orig)

        img = Image.open(ruta_orig)
        angle = random.choice(angles)
        img_rot = img.rotate(angle, expand=True)

        base, ext = os.path.splitext(nom_orig)
        nou_nom = f"{base}_rot{angle}_{generades}{ext.lower()}"
        img_rot.save(os.path.join(cls_dir, nou_nom))
        generades += 1

    print(f"{os.path.basename(cls_dir)}: {generades} imatges creades")

classes = os.listdir(train_dir)

for cls in classes:
    augmentar_classe(os.path.join(train_dir, cls), augment_per_class)

print("Fet")

### 3. Creació dels Generadors de Dades
Utilitzem ImageDataGenerator per generar dades augmentades de les imatges d'entrenament i validació. També aplicarem la funció de pre-processament adequada per als models InceptionV3 i MobileNetV2.

In [ ]:
def crear_generadors(dataset_path, target_size, preprocess_func):
    gen_train = ImageDataGenerator(preprocessing_function=preprocess_func)
    gen_val = ImageDataGenerator(preprocessing_function=preprocess_func)

    train_data = gen_train.flow_from_directory(
        os.path.join(dataset_path, "train"),
        target_size=target_size,
        batch_size=32,
        class_mode="binary"
    )

    val_data = gen_val.flow_from_directory(
        os.path.join(dataset_path, "val"),
        target_size=target_size,
        batch_size=32,
        class_mode="binary",
        shuffle=False
    )
    return train_data, val_data

def train_model(base_model, optimizer, model_name, train_data, val_data, epochs=50):
    base_model.trainable = False
    # definir model
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation="relu"),
        Dense(1, activation="sigmoid")
    ])
    # configurar model
    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)
    # entrenar model
    history = model.fit(train_data, epochs=epochs, validation_data=val_data, callbacks=[early_stop])
    model.save(f"{model_name}.h5")
    return model, history

## 4. Entrenament del Model
Per entrenar els models utilitzem les xarxes preentrenades InceptionV3 i MobileNetV2. Els models tenen una finestra de paciència de 3 epochs per evitar l'overfitting.

In [ ]:
dataset_path = "dataset/"
# Model 1 - InceptionV3
inceptionv3 =  applications.InceptionV3(include_top=False, input_shape=(299, 299, 3), weights="imagenet")
train_data_i, val_data_i = crear_generadors(dataset_path, target_size=(299, 299), preprocess_func=preprocess_incep)
model_incepv3, history_incepv3 = train_model(inceptionv3, RMSprop(learning_rate=0.0001), "model_inceptionv3_50", train_data_i, val_data_i)

# Model 2 - MobileNetV2
mobilenetv2 = applications.MobileNetV2(include_top=False, input_shape=(224, 224, 3), weights="imagenet")
train_data_m, val_data_m = crear_generadors(dataset_path, target_size=(224, 224), preprocess_func=preprocess_mobile)
model_mobilenetv2, history_mobilenetv2 = train_model(mobilenetv2, Adam(learning_rate=0.0001), "model_mobilenetv2_50", train_data_m, val_data_m)

### 5. Avaluació del Model
Després de l'entrenament, avaluem el rendiment dels models utilitzant les mètriques de precisió i pèrdua tant per al conjunt d'entrenament com de validació. També analitzem el comportament dels models utilitzant diferents llindars de classificació.

In [ ]:
model_histories = [
    ("MobileNetV2", history_mobilenetv2),
    ("InceptionV3", history_incepv3)
]

for model_name, history in model_histories:
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))

    # Gràfica de precisió
    plt.figure(figsize=(8, 6))
    plt.plot(epochs, acc, 'b', label='Training Accuracy')
    plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
    plt.title(f'{model_name} - Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Gràfica de pèrdua
    plt.figure(figsize=(8, 6))
    plt.plot(epochs, loss, 'b', label='Training Loss')
    plt.plot(epochs, val_loss, 'r', label='Validation Loss')
    plt.title(f'{model_name} - Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
def evaluate_model_thresholds(model, val_data, model_name, thresholds=[0.2, 0.3, 0.4, 0.5]):
    probs = model.predict(val_data)
    true_labels = val_data.classes[:len(probs)]

    for threshold in thresholds:
        preds = (probs > threshold).astype(int).flatten()
        print(f"\nClassificació per llindar {threshold} - {model_name}:")
        print(classification_report(true_labels, preds, target_names=val_data.class_indexs.keys()))

        cm = confusion_matrix(true_labels, preds)
        plt.figure(figsize=(6, 6))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                    xticklabels=val_data.class_indexs.keys(),
                    yticklabels=val_data.class_indexs.keys())
        plt.title(f"Matriu de confusió - {model_name} (llindar: {threshold})")
        plt.ylabel("Etiqueta real")
        plt.xlabel("Predicció")
        plt.show()

thresholds = [0.2, 0.3, 0.4, 0.5]

evaluate_model_thresholds(model_incepv3, val_data_i, "InceptionV3", thresholds)
evaluate_model_thresholds(model_mobilenetv2, val_data_m, "MobileNetV2", thresholds)

### 6. Visualització d'Errors
Visualitzem els errors (falsos positius i falsos negatius) per diferents llindars per entendre millor el comportament dels models en situacions d'error.

In [ ]:
def errors_per_thresholds(model, val_data, class_indexs, thresholds=[0.2, 0.3, 0.4, 0.5], max_mostres=10):
    true_labels = val_data.classes
    class_names = list(class_indexs.keys())
    probs = model.predict(val_data, verbose=0)
    file_paths = val_data.filepaths
    idx_to_class = {v: k for k, v in class_indexs.items()}

    for threshold in thresholds:
        preds = (probs > threshold).astype(int).flatten()
        fp_indexs = np.where((preds == 1) & (true_labels == 0))[0]
        fn_indexs = np.where((preds == 0) & (true_labels == 1))[0]

        print(f"\nLlindar: {threshold}")

        def mostrar_muestras(indexs, titol):
            print(f"\n{titol} ({len(indexs)} mostres):")
            for i in indexs[:max_mostres]:
                img_path = file_paths[i]
                img = plt.imread(img_path)
                base_name = os.path.basename(img_path)
                plt.imshow(img)
                plt.axis("off")
                plt.title(f"{base_name}\nReal: {idx_to_class[true_labels[i]]} | Predicció: {idx_to_class[preds[i]]}")
                plt.show()

        mostrar_muestras(fp_indexs, "Falsos positius (Prediu 'defecte', era 'nodefecte')")
        mostrar_muestras(fn_indexs, "Falsos negatius (Prediu 'nodefecte', era 'defecte')")

thresholds = [0.2, 0.3, 0.4, 0.5]

print("\nErrors InceptionV3:")
errors_per_thresholds(model_incepv3, val_data_i, val_data_i.class_indexs, thresholds)

print("\nErrors MobileNetV2:")
errors_per_thresholds(model_mobilenetv2, val_data_m, val_data_m.class_indexs, thresholds)

### 7. Predicció i visualització de resultats
Finalment, es poden visualitzar les prediccions del model sobre un conjunt d'imatges de validació, juntament amb la confiança en la predicció.

In [ ]:
models = {"InceptionV3": load_model("model_inceptionv3_50.h5")}

class_indexs = {v: k for k, v in val_data_i.class_indexs.items()}

mostrar_img = 30
lote_test, etiquetes_reals = next(val_data_i)

for i in range(min(mostrar_img, len(lote_test))):
    plt.imshow(lote_test[i])
    plt.axis("off")
    plt.show()

    img_array = np.expand_dims(lote_test[i], axis=0)

    for nombre, model in models.items():
        prob = float(model.predict(img_array, verbose=0)[0][0])
        classe_pred = int(prob >= 0.3)
        nom_classe_pred = class_indexs[classe_pred]
        nom_classe_real = class_indexs[int(etiquetes_reals[i])]
        confiança = prob if classe_pred == 1 else 1 - prob

        print(f"{nombre}: Predicció → {nom_classe_pred} (Confiança: {confiança * 100:.2f}%) | Real: {nom_classe_real}")